# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Análise Estatística Para Data Science com Linguagem Python</font>
## <font color='blue'>Mini-Projeto 5</font>
## <font color='blue'>Market Basket Analysis Para Recomendações de Compras aos Clientes</font>

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.18


In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [2]:
!pip install -q mlxtend


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# Imports
import pandas as pd
import mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

pandas : 2.0.3
mlxtend: 0.23.1



## Carregando e Compreendendo os Dados

In [5]:
# Exemplo de transações
dados = [
    ['Leite', 'Pão', 'Café'],
    ['Leite', 'Biscoitos'],
    ['Pão', 'Ovos', 'Manteiga'],
    ['Leite', 'Ovos', 'Café'],
    ['Leite', 'Pão', 'Biscoitos', 'Café']
]

O TransactionEncoder é uma classe da biblioteca mlxtend, que é frequentemente usada em análises de dados, especialmente em tarefas de mineração de dados como a análise de regras de associação. Essa classe é especificamente projetada para transformar dados de transação - tipicamente listas de listas contendo itens de transação - em um formato adequado para algoritmos de mineração de regras de associação, como o Apriori.

In [6]:
# Prepara o objeto
transaction_enc = TransactionEncoder()

In [7]:
# Treina o encoder e aplica aos dados, gerando as transações
transactions = transaction_enc.fit(dados).transform(dados)

Aqui está um resumo do que o TransactionEncoder faz:

**Transformação de Dados**: Converte os dados de transações, onde cada transação é uma lista de itens, em um formato de matriz binária. Cada coluna representa um item único presente no conjunto de dados e cada linha representa uma transação.

**Codificação Binária**: Em cada linha (transação), '1' (ou True) indica a presença do item naquela transação, enquanto '0' (ou False) indica a ausência do item.

**Facilita a Análise**: Este formato de dados é ideal para a análise de regras de associação, pois permite que algoritmos como o Apriori percorram facilmente o conjunto de dados para encontrar padrões frequentes, regras de associação, ou conjuntos de itens frequentes.

**Inversão de Dados**: O TransactionEncoder também pode reverter o processo, convertendo a matriz binária de volta para o formato de lista de transações, o que pode ser útil para visualização ou outras formas de análise.

In [8]:
transactions

array([[False,  True,  True, False, False,  True],
       [ True, False,  True, False, False, False],
       [False, False, False,  True,  True,  True],
       [False,  True,  True, False,  True, False],
       [ True,  True,  True, False, False,  True]])

In [9]:
# Converte as transações em dataframe com os nomes das colunas
df = pd.DataFrame(transactions, columns = transaction_enc.columns_)

In [10]:
df.head()

,Biscoitos,Café,Leite,Manteiga,Ovos,Pão
0,False,True,True,False,False,True
1,True,False,True,False,False,False
2,False,False,False,True,True,True
3,False,True,True,False,True,False
4,True,True,True,False,False,True


## Algoritmo Apriori

O Apriori é um algoritmo clássico na mineração de dados usado para extrair conjuntos de itens frequentes para regras de associação booleanas. Ele opera em um conjunto de dados de transações, procurando combinações de itens que aparecem juntos com frequência.

In [11]:
# Aplicando o algoritmo Apriori
frequent_itemsets = apriori(df, min_support = 0.2, use_colnames = True)

Descrição dos Parâmetros:

**df**: Este é o DataFrame que você está passando para o algoritmo Apriori. Este DataFrame deve estar no formato de uma matriz binária, onde cada coluna representa um item diferente e cada linha representa uma transação, com 1 (ou True) indicando a presença do item na transação, e 0 (ou False) indicando a ausência. Esta transformação geralmente é feita usando o TransactionEncoder.

**min_support = 0.2**: Este é o valor de suporte mínimo para ser considerado um conjunto de itens frequentes. O suporte de um conjunto de itens é definido como a proporção de transações no conjunto de dados que contêm o conjunto de itens. Neste caso, 0.2 significa que um conjunto de itens precisa aparecer em pelo menos 20% de todas as transações para ser considerado frequente.

**use_colnames = True**: Este argumento indica que o algoritmo deve usar os nomes das colunas do DataFrame em vez de índices inteiros para os itens. Isso torna a saída mais legível, pois você verá os nomes dos itens reais nos conjuntos de itens frequentes, em vez de números representando suas posições no DataFrame.

**frequent_itemsets**: Este é o DataFrame de saída que contém todos os conjuntos de itens que atendem ao critério de suporte mínimo. Cada linha representa um conjunto de itens frequentes, juntamente com seu respectivo valor de suporte.

## Regras de Associação

In [12]:
type(frequent_itemsets)

pandas.core.frame.DataFrame

In [13]:
frequent_itemsets.head(10)

,support,itemsets
0,0.4,(Biscoitos)
1,0.6,(Café)
2,0.8,(Leite)
3,0.2,(Manteiga)
4,0.4,(Ovos)
5,0.6,(Pão)
6,0.2,"(Biscoitos, Café)"
7,0.4,"(Biscoitos, Leite)"
8,0.2,"(Pão, Biscoitos)"
9,0.6,"(Café, Leite)"


In [14]:
# Geração de regras de associação
rules = association_rules(frequent_itemsets, metric = "confidence", min_threshold = 0.5)

Descrição dos Parâmetros:

**association_rules**: Esta função é parte da biblioteca mlxtend usada para mineração de regras de associação. Ela pega os conjuntos de itens frequentes gerados por um algoritmo como o Apriori e gera regras de associação a partir deles.

**frequent_itemsets**: Este é o DataFrame contendo os conjuntos de itens frequentes, que é a saída do algoritmo Apriori. Cada conjunto de itens frequentes neste DataFrame foi identificado como aparecendo frequentemente no conjunto de dados, com base no valor de suporte mínimo fornecido anteriormente.

**metric = "confidence"**: Este argumento especifica a métrica a ser usada para avaliar a força de uma regra de associação. Neste caso, estamos usando a "confiança" como métrica. A confiança de uma regra é uma medida de quão frequentemente os itens no lado direito da regra aparecem em transações que contêm os itens no lado esquerdo. É calculada como a proporção do número de transações com todos os itens na regra pelo número de transações com os itens no lado esquerdo da regra.

**min_threshold = 0.5**: Este argumento estabelece o valor mínimo da métrica escolhida (neste caso, a confiança) para uma regra ser considerada significativa. Aqui, 0.5 significa que apenas as regras com uma confiança de 50% ou mais serão incluídas. Em outras palavras, para cada regra gerada, pelo menos metade das vezes que os itens do lado esquerdo aparecem, os itens do lado direito também devem aparecer.

**rules**: Este é o DataFrame resultante que contém as regras de associação. Cada linha neste DataFrame representa uma regra de associação e incluirá colunas para itens do lado esquerdo (antecedente), itens do lado direito (consequente), e métricas de interesse como suporte, confiança, e lift.

In [15]:
type(rules)

pandas.core.frame.DataFrame

In [16]:
rules.head(30)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Biscoitos),(Café),0.4,0.6,0.2,0.500000,0.833333,-0.04,0.8,-0.250000
1,(Biscoitos),(Leite),0.4,0.8,0.4,1.000000,1.250000,0.08,inf,0.333333
2,(Leite),(Biscoitos),0.8,0.4,0.4,0.500000,1.250000,0.08,1.2,1.000000
3,(Biscoitos),(Pão),0.4,0.6,0.2,0.500000,0.833333,-0.04,0.8,-0.250000
4,(Café),(Leite),0.6,0.8,0.6,1.000000,1.250000,0.12,inf,0.500000
5,(Leite),(Café),0.8,0.6,0.6,0.750000,1.250000,0.12,1.6,1.000000
6,(Ovos),(Café),0.4,0.6,0.2,0.500000,0.833333,-0.04,0.8,-0.250000
7,(Pão),(Café),0.6,0.6,0.4,0.666667,1.111111,0.04,1.2,0.250000
8,(Café),(Pão),0.6,0.6,0.4,0.666667,1.111111,0.04,1.2,0.250000
9,(Ovos),(Leite),0.4,0.8,0.2,0.500000,0.625000,-0.12,0.4,-0.500000


Esses resultados são métricas utilizadas na análise de regras de associação, uma técnica comum em análise de cestas de mercado (Market Basket Analysis). Vamos definir cada um deles:

**Antecedents**: Os antecedentes em uma regra de associação são os itens que antecedem a consequência. Por exemplo, na regra {Leite} -> {Pão}, "Leite" é o antecedente.

**Consequents**: Os consequentes são os itens que vêm após os antecedentes na regra de associação. No exemplo {Leite} -> {Pão}, "Pão" é o consequente.

**Antecedent Support**: O suporte do antecedente é a proporção de transações no conjunto de dados que contém o antecedente. Por exemplo, se "Leite" aparece em 30% das transações, então o suporte de "Leite" é 0,30.

**Consequent Support**: O suporte do consequente é a proporção de transações que contém o consequente. Se "Pão" aparece em 40% das transações, então o suporte de "Pão" é 0,40.

**Support**: O suporte da regra é a proporção de transações que contêm todos os itens na regra. Por exemplo, se "Leite" e "Pão" aparecem juntos em 20% das transações, então o suporte da regra {Leite} -> {Pão} é 0,20.

**Confidence**: A confiança é uma medida de quão frequentemente a regra é verdadeira. Em outras palavras, dentre todas as transações que contêm os antecedentes, qual proporção também contém os consequentes. Por exemplo, se 70% das transações que contêm "Leite" também contêm "Pão", a confiança da regra {Leite} -> {Pão} é 0,70.

**Lift**: O lift é uma medida de quão mais provável é que os consequentes sejam comprados quando os antecedentes são comprados, comparado com a frequência de compra dos consequentes em geral. Um lift maior que 1 indica uma associação positiva.

**Leverage**: A alavancagem mede a diferença na frequência de ocorrência dos antecedentes e consequentes juntos na base de dados e o que seria esperado se fossem independentes. Valores positivos indicam uma associação positiva.

**Conviction**: A convicção é uma medida de dependência entre antecedentes e consequentes. Um valor alto de convicção significa que é altamente provável que os consequentes sejam comprados quando os antecedentes são comprados.

**Zhang's Metric**: Esta é uma medida de correlação entre antecedentes e consequentes. Varia entre -1 e 1, onde 1 significa correlação positiva perfeita, 0 nenhuma correlação e -1 correlação negativa perfeita.

# Fim